In [446]:
%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler

In [447]:
X_raw = np.random.random(100*9)
X_raw = np.reshape(X_raw, (100,9))

In [448]:
scaler = StandardScaler().fit(X_raw)

In [449]:
X = scaler.transform(X_raw)

In [450]:
#Add an intercept column to the model
X = np.abs(np.concatenate((np.ones((X.shape[0], 1)), X), axis=1))

In [451]:
beta = np.array([0.5,0.6,0.7,0.3,0.5,0.7,0.1,0.2,0.2,0.8])

In [463]:
Y = np.matmul(X, beta)

In [462]:
class Elasso:
    """
    This solves the customized linear model I proposed.
    """
    def __init__(self,X, y, lambda1=1, lambda2=1, max_iter=1000):
        """
        Coordiante descent algorithm (cyclic) to solve the customized linear model.
        
        Parameters:
        ----------
        X: matrix (n by k)
        The exogenous variables for Elasso.
        
        y: vector (n by 1)
        The endogenoud variable for Elasso.
        
        lambda_1: float
        Equality regularization parameter.
        The default lambda_1 is set to 1. However, users are allowed to set lambda_1 in the __main__ function 
        to see different training performances.
        
        lambda_2: float
        Sparsity regularization parameter.
        The default lambda_2 is set to 1. However, users are allowed to set lambda_2 in the __main__function.
        
        max_iter: int
        Default is set as 1000.
        However, users are allowed to set max_iter in the __main__ function to see different performances.
        
        beta: vector
        This is the estimated beta computed from the coordinate descent algorithm using the brute force 
        way though.
        """
        #Validate inputs
        if X.shape[0] != y.shape[0]:
            raise ValueError("The input size does not match")
        # Initialize class instance variables:
        self.X = X
        self.y = y
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.max_iter = max_iter
        self.beta = self.coordinatedescent()
        
    def customizedloss(self, beta):
        """
        This is a customized loss function for the customized LASSO function in my research, where lambda1 is the hyperparameter for
        the equality constraint where lambda2 is the constraint for sparsity.
        """
        penalty1 = 0
        for i in range(len(beta)-1):
            penalty1 += abs(beta[i] - beta[i+1])
        penalty1 = penalty1*self.lambda1
        penalty2 = sum([abs(b) for b in beta])
        penalty2 = self.lambda2*penalty2
        #print("penalty2 is %s" %penalty2)
        y_true = np.array(self.y)
        y_pred = np.array(np.matmul(self.X, beta))
        assert len(y_true) == len(y_pred)
        fittingloss = np.sum((y_true - y_pred)**2)
        totalloss = fittingloss + penalty1 + penalty2
        return totalloss
    
    def coordinatedescent(self):
        # Initialization 
        intbeta = [0] * self.X.shape[1]
        # This step I split the 
        Range = np.linspace(-0.9999, 0.9999, 1000)
        num = 0
        oldbeta = intbeta.copy()
        newbeta = intbeta.copy()
        while num < self.max_iter:
            for i in range(len(intbeta)):
                loss = []
                #print("This is the original new beta, %s" % newbeta)
                for x in Range:
                    betatemp = newbeta.copy()
                    betatemp[i] = x
                    value = self.customizedloss(betatemp)
                    loss.append(value)
                minIndex = loss.index(min(loss))
                newbeta[i] = Range[minIndex]
                #print("This is the new beta after minimization, %s" % newbeta)
            mse = calmse(oldbeta, newbeta)
            #print("This is the old beta before substitution, %s" % oldbeta)
            oldbeta = newbeta.copy()
            #print("This is the old beta after substitution, %s" % oldbeta)
            num += 1
            print("This is the mse, %s" %mse)
            if mse < 0.00000001:
                return newbeta
                break
                print("Maximum iteration reached")
        return newbeta
    
def calmse(oldbeta, newbeta):
    mse = 0
    for old, new in zip(oldbeta, newbeta):
        mse +=(old - new)**2
    return mse

In [460]:
model = Elasso(X, Y, lambda1=0, lambda2=0)

This is the mse, 4.334898084942781
This is the mse, 0.10326982060222391
This is the mse, 0.06564211444200964
This is the mse, 0.04388696888888884
This is the mse, 0.03134440016881747
This is the mse, 0.024451998188458703
This is the mse, 0.020152261371641954
This is the mse, 0.017158875162730285
This is the mse, 0.01471046957552147
This is the mse, 0.012462424510997449
This is the mse, 0.01032658133917704
This is the mse, 0.008411134742309872
This is the mse, 0.0064636304618131656
This is the mse, 0.0048206741757974144
This is the mse, 0.003570424514243971
This is the mse, 0.002580643532180841
This is the mse, 0.0017391293369044612
This is the mse, 0.0013063506079052057
This is the mse, 0.0009216584043502923
This is the mse, 0.0007453411443876354
This is the mse, 0.0007733916175635066
This is the mse, 0.0006812257771284812
This is the mse, 0.0008855935102670254
This is the mse, 0.000805449301193088
This is the mse, 0.0008855935102670231
This is the mse, 0.0010018026134242346
This is th

In [458]:
beta

array([0.5, 0.6, 0.7, 0.3, 0.5, 0.7, 0.1, 0.2, 0.2, 0.8])

In [391]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(X, Y)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [99]:
import statsmodels.api as sm

In [100]:
model = sm.OLS(Y_true, X)